In [6]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} scikit-image

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\gunne\anaconda3\envs\cs791

  added / updated specs:
    - scikit-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.20.1               |       h7bd577a_0         142 KB
    brotli-1.0.9               |       ha925a31_2         332 KB
    charls-2.1.0               |       h33f27b4_2          92 KB
    cloudpickle-1.6.0          |             py_0          30 KB
    cytoolz-0.11.0             |   py38he774522_0         289 KB
    dask-core-2020.12.0        |     pyhd3eb1b0_0         632 KB
    giflib-5.2.1               |       h62dcd97_0          81 KB
    imagecodecs-2020.5.30      |   py38hb1be65f_2         5.7 MB
    imageio-2.9.0              |             py_0         3.0 MB
    lcms2-2.11                 |       hc51a39a_0         451 KB
    libaec-1.0.4               |      

In [10]:
import numpy as np
import tensorflow as tf
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import cv2
import random
from random import seed
from random import randint
from numpy import savetxt
from PIL import Image
import os
from skimage import data, transform
from tqdm.notebook import tqdm

## Preprocess the Fashion-MNIST data

In [11]:
np.random.seed(1)
tf.random.set_seed(1)
batch_size = 128

#Load fashion MNIST and store into array training_features (dont load in labels)
(training_features, _), _ = tf.keras.datasets.fashion_mnist.load_data()
#normalize pixel values
training_features = training_features / np.max(training_features)

training_dataset = tf.data.Dataset.from_tensor_slices(training_features)
training_dataset = training_dataset.batch(batch_size)
#dont shuffle
#training_dataset = training_dataset.shuffle(training_features.shape[0])
training_dataset = training_dataset.prefetch(batch_size * 4)

## NonRigid Transformation Function Definitions

In [12]:
# From https://mmas.github.io/linear-transformations-numpy
def linear_transformation(src, a):
    M, N = src.shape
    points = np.mgrid[0:N, 0:M].reshape((2, M*N))
    new_points = np.linalg.inv(a).dot(points).round().astype(int)
    x, y = new_points.reshape((2, M, N), order='F')
    indices = x + N*y
    return np.take(src, indices, mode='wrap')

# From https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478
def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

def shearImage(img,sx,sy):
    trans = transform.AffineTransform(shear=-sx)
    img = transform.warp(img, trans, order=1, preserve_range=True, mode='constant')

    trans = transform.AffineTransform(shear=-sy)
    img = np.swapaxes(img, 0, 1)
    img = transform.warp(img, trans, order=1, preserve_range=True, mode='constant')
    img = np.swapaxes(img, 0, 1)

    return img

## Generate NonRigid Images

#### Generates 60,000 Original/Transformed/Composite

In [14]:
def generateNonRigid():
    transforms = randint(1,3)
    #progress bar for tqdm
    pbar = tqdm(range(len(training_features)))
    for i in pbar:
        transformation = training_features[i]
        if transforms == 3:
            # zoom
            # assign the zooms equal to eachother
            zoom = 1+random.random()
            transformation = clipped_zoom(transformation,zoom)
            # sheer
            degree = random.random()
            transformation = shearImage(transformation,sx=degree,sy=degree)
        elif transforms == 2:
            # zoom
            # assign the zooms equal to eachother
            zoom = 1+random.random()
            transformation = clipped_zoom(transformation,zoom)
        elif transforms == 1:
             # sheer
            degree = random.random()
            transformation = shearImage(transformation,sx=degree,sy=degree)
        
        composite = (training_features[i]/2) + (transformation/2)
        
        #create image objects of transformation/original/composite
        transformation_img = Image.fromarray(np.uint8(transformation*255),'L')
        original_img = Image.fromarray(np.uint8(training_features[i]*255),'L')
        composite_img = Image.fromarray(np.uint8(composite*255),'L')
        
        #save images to disk
        transformation_img.save('C:\\Users\\gunne\\Desktop\\CS791\\NonRigid\\Transformed\\'+str(i)+'.jpg')
        original_img.save('C:\\Users\\gunne\\Desktop\\CS791\\NonRigid\\Original\\'+str(i)+'.jpg')
        composite_img.save('C:\\Users\\gunne\\Desktop\\CS791\\NonRigid\\Composite\\'+str(i)+'.jpg')


if __name__ == "__main__":
    generateNonRigid()